<p style="text-align:justify">&emsp;&emsp;Proses pelatihan (penyesuaian nilai bobot dan bias) dengan menggunakan metode propagasi balik dapat memakan waktu karena sifatnya yang iterarif(<a href="https://www.semanticscholar.org/paper/Extreme-learning-machine%3A-a-new-learning-scheme-of-Huang-Zhu/2b9c0e4d1d473aadbe1c2a76f75bc02bfa6416b0">baca</a>). Oleh karena itu saya mencoba menerapkan metode pelatihan yang sifatnya yang tidak iteratif(sekali jalan/<i>feed forward</i>), cepat, dan juga mengahsilkan akurasi yang tidak kalah tinggi dengan metode propagasi balik, yaitu <i>Extreme Learning Machine</i>(ELM). Pada dasarnya ELM hanya memiliki satu <i>hidden layer</i> dengan jumlah node yang dapat disesuaikan.</p>
<img src="https://www.researchgate.net/publication/265608741/figure/fig3/AS:213795097059335@1427984131066/The-structure-of-extreme-learning-machine.png"/>
Pada ELM bobot antara <i id="rumus">input layer</i> dan <i>hidden layer</i> dibuat secara acak(antara 1-0).<br>
Bobot yang disesuaikan(dilatih) adalah bobot antara <i>hidden layer</i> dan <i>output layer</i>.<br>
Berikut adalah persamaan fungsi untuk mencari nilai bobot(disimbolkan beta) antara <i>hidden layer</i> dan <i>output layer</i> :<br>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/3c23b9696f0b9c900b68e71fc63b1b5a0e0cc1e1"/><br>
X adalah nilai masukan<br>
T adalah transpose<br>
y adalah nilai target.

# Impor <i>libraries</i>

In [1]:
import numpy as np
from pandas import read_csv

from sklearn.model_selection import train_test_split

NumPy buat operasi matriks<br>
read_csv dari Pandas buat muat <i>dataset</i><br>
train_test_split buat membagi data uji dan data latih<br>

# Muat <i>dataset</i>

In [2]:
data = read_csv("seeds_data.csv")
data

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,class
0,15.2,14.8,0.870,5.76,3.31,2.22,5.20,1
1,14.8,14.5,0.881,5.55,3.33,1.01,4.95,1
2,14.2,14.0,0.900,5.29,3.33,2.69,4.82,1
3,13.8,13.9,0.895,5.32,3.37,2.25,4.80,1
4,16.1,14.9,0.903,5.65,3.56,1.35,5.17,1
...,...,...,...,...,...,...,...,...
205,12.1,13.0,0.878,5.13,2.98,3.63,4.80,3
206,11.2,12.8,0.851,5.10,2.79,4.32,5.00,3
207,13.0,13.6,0.888,5.23,3.23,8.31,5.05,3
208,11.8,13.2,0.852,5.17,2.83,3.59,5.04,3


Datanya diambil dari sini : https://archive.ics.uci.edu/ml/datasets/seeds<br>
Sudah dibersihkan secara manual untuk mengisi nilai yang kosong dengan metode <i>simple rolling mean</i>

# Tentukan data latih dan label yang jadi acuan klasifikasi

In [3]:
x_train = data.drop(axis=1, columns='class')

labels = data['class']

Seluruh data kecuali kolom <b>class</b> ditampung ke dalam variabel <b>x_train</b><br>
Sedangkan data pada kolom <b>class</b> ditampung ke dalam variabel <b>labels</b>

In [4]:
x_train.head()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove
0,15.2,14.8,0.870,5.76,3.31,2.22,5.20
1,14.8,14.5,0.881,5.55,3.33,1.01,4.95
2,14.2,14.0,0.900,5.29,3.33,2.69,4.82
3,13.8,13.9,0.895,5.32,3.37,2.25,4.80
4,16.1,14.9,0.903,5.65,3.56,1.35,5.17


In [5]:
labels.ravel()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

### Lakukan <i>one hot encoding</i>

In [6]:
kelas = len(np.unique(labels))

y_train = np.zeros([len(labels), kelas])

for i in range(len(labels)):
        if i < len(labels)/kelas:
            y_train[i][0] = 1
        elif i > len(labels)/kelas and i < len(labels)/kelas*2:
            y_train[i][1] = 1
        else:
            y_train[i][2] = 1

Nah jadi <i>one hot encoding</i> ini sebetulnya cuma ngubah format label aja sih, biar enak waktu dihitung.<br>
Contohnya seperti ini :

In [7]:
print(y_train.shape)

y_train

(210, 3)


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

Nah itu total barisnya ada 210 sama seperti todal baris pada data.<br>
Kemudian memiliki tiga kolom di mana masing-masing kolom mewakili nilai kelas dari tiap data.<br>
Pada baris ke 1-70 itu masuk ke dalam kelas 1. Jadi angka satu berdada di indeks 0.<br>
Baris ke 71-140 masuk kedalam kelas 2. Angka satu berada pada indeks ke 1.<br>
Baris ke 141-210 masuk kedalam kelas 3 dan angka satu berada pada indeks ke 2.<br><br>
Saya rasa ini mudah dipahami hehehe

# Pisah data uji dan data latih

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3)

print("Data latih :",len(x_train))
print("Data uji   :", len(x_test))

Data latih : 147
Data uji   : 63


Oke kita jadikan 70% data sebagai data latih dan sisanya 30% jadi data uji

# Inisialisasi fungsi yang digunakan untuk menghitung

In [9]:
# FUNGSI AKTIVASI
def sigmoid(x):
    return 1/(1 + np.exp(-x))

<img src="https://qph.fs.quoracdn.net/main-qimg-05edc1873d0103e36064862a45566dba"/><br>
Pada percobaan kali ini saya menggunakan fungsi aktivasi sigmoin di mana nilai yang dihasilkan adalah dari rentang 0 sampai 1.<br>

In [10]:
def input_to_hidden(x):
    a = np.dot(x, Win)
    a += bias
    a = sigmoid(a)
    return a

Fungsi di atas adalah untuk proses perkalian nilai masukan pada bobot yang ada pada antara <i>input layer</i> dan <i>hidden layer</i>

In [11]:
def predict(x, bobot):
    x = input_to_hidden(x)
    y = np.dot(x, bobot)      # PERHITUNGAN DARI HIDDEN LAYER MENUJU OUTPUT LAYER
    return y

Fungsi di atas itu untuk memprediksi nilai masukan atau data uji ya.<br>
Nah jadi kita panggil fungsi <b>input_to_hidden</b> kemudian nilai kembaliannya dimasukkan ke variabel <b>x</b>.<br>
Setelah itu nilai yang ada pada variabel <b>x</b> dikalikan secara matriks dengan parameter <b>bobot</b> di mana parameter ini menampung nilai perhitungan antara <i>hidden layer</i> dan <i>output layer</i>.

# Inisialisasi <i>input layer</i> dan <i>hidden layer</i>

In [12]:
inpt = x_train.shape[1]   # 7 NODE PADA INPUT LAYER
hidden = 100             # 100 NODE PADA HIDDEN LAYER

bias = np.random.randn(hidden)
Win = np.random.normal(size=[inpt, hidden])

Win.shape

(7, 100)

Jaringan saraf tiruan yang kita susun terdiri dari 7 node <i>input layer</i> dan 100 node <i>hidden layer</i>.<br>
"Kenapa kok 7?" Ya karena nilai <i>input</i> untuk mengklasifikasikan nilai pada kolom <b>class</b> jumlahnya ada 7.<br>
Jumlah node pada <i>hidden layer</i> kok 100 itu sebenernya saya ndak ada kriteria khusus sih, saya cuma mau masang nilai 100 aja heheheh (walaupun sebetulnya sudah pernah dicoba dengan nilai-nilai yang lain).<br>

"Lah terus jumlah node pada output layer'nya mana nih?<br>
Nggak diatur sekalian?"<br>

Itu nanti terbentuk setelah melalui perhitungan pake rumus di <a href="#rumus">atas</a>.

# Perhitungan ELM

In [13]:
X = input_to_hidden(x_train)
Xt = np.transpose(X)

Wout = np.dot(np.linalg.pinv(np.dot(Xt, X)), np.dot(Xt, y_train))   # BOBOT YANG SUDAH TERLATIH

Wout.shape

(100, 3)

Kita panggil fungsi <b>input_to_hidden</b> dengan <b>x_train</b> sebagai argumen kemudian nilai kembaliannya ditampung ke dalam variabel <b>X</b>.<br>
Nilai transpose dari variabel <b>X</b> disimpan ke dalam variabel <b>Xt</b>.<br>

Nah jadi perhitungan bobot selanjutnya (dari <i>hidden layer</i> ke <i>output layer</i>) itu pake rumus ini:<br>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/3c23b9696f0b9c900b68e71fc63b1b5a0e0cc1e1"/><br>
Nanti nilainya ditampung ke dalam variabel <b>Wout</b> yang memiliki matriks berukuran 100x3.<br>

Sebetulnya fungsi yang digunakan adalah <b>np.linalg.inv</b> bukan <b>np.linalg.pinv</b>.<br>
Saya menggunakan yang <b>np.linalg.pinv</b> karena matriks yang terbentuk dari hasil menggunakan <b>np.linalg.inv</b> (operasi invers) adalah matriks singular.<br>
Solusi dari Stackoverflow itu saya kudu menggunakan <a href="https://en.wikipedia.org/wiki/Singular_value_decomposition">singular value decomposition</a> yang sudah ada pada fungsi <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.pinv.html"><b>np.linalg.pinv</b></a>.<br>
Arti <b>p</b> dari <b>.pinv</b> adalah pseudo.

Di mana 100 itu jumlah node pada <i>hidden layer</i> dan 3 itu adalah jumlah node pada <i>output layer</i>.<br>
Nah sekarang jadi tahu kan gimana <i>output layer</i> terbentuk heheheh...<br>

# Evaluasi ELM

Karena bobot sudah pada dilatih semua yaudah saatnya kita uji akurasi ELM kita

In [15]:
y = predict(x_test, Wout)

betul = 0
total = len(y)

for i in range(total):
    hasil_prediksi = np.argmax(y[i])
    tes = np.argmax(y_test[i])
    betul = betul + (1 if hasil_prediksi == tes else 0)

print("Akurasi:", betul/total)

Akurasi: 0.9682539682539683


Alhamdulillah ELM yang dirancang punya akurasi di atas 95% hehehe.<br>
Ini nih kalau mau lihat nilai hasil prediksi dan nilai yang asli

In [16]:
np.argmax(y, axis=1)

array([1, 2, 0, 1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 2, 0, 0,
       1, 0, 2, 1, 1, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 2, 1, 2, 2, 1, 1, 2, 0, 0, 0, 0, 0])

In [17]:
np.argmax(y_test, axis=1)

array([1, 2, 0, 1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 2, 0, 0,
       1, 0, 2, 1, 1, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0,
       0, 0, 1, 0, 0, 2, 0, 0, 1, 2, 2, 1, 1, 2, 0, 0, 0, 0, 0])